In [6]:
from scipy.io import loadmat
mat = loadmat('./PIE dataset/Pose05_64x64.mat')

In [7]:
mat

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon May 08 13:10:30 2017',
 '__version__': '1.0',
 '__globals__': [],
 'isTest': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]),
 'fea': array([[ 8,  9,  9, ..., 35, 36, 42],
        [ 6,  7,  7, ..., 38, 32, 31],
        [ 8,  8,  6, ..., 26, 25, 23],
        ...,
        [11, 10,  8, ..., 36, 40, 42],
        [ 9, 11,  9, ..., 29, 31, 36],
        [12,  9,  9, ..., 29, 34, 38]], dtype=uint8),
 'gnd': array([[ 1],
        [ 1],
        [ 1],
        ...,
        [68],
        [68],
        [68]], dtype=uint8)}

In [8]:
mat['fea'].shape

(3332, 4096)

In [9]:
mat['gnd'].shape

(3332, 1)

In [53]:
import pandas as pd
X = pd.DataFrame(data=mat['fea'])
y = pd.DataFrame(data=mat['gnd'])
isTest = pd.DataFrame(data=mat['isTest'])
X.sample(frac=1, random_state=0)
y.sample(frac=1, random_state=0)
isTest.sample(frac=1, random_state=0)
X.shape

(3332, 4096)

In [55]:
X_train = X[isTest[0] == 0]
X_test = X[isTest[0] == 1]
y_train = y[isTest[0] == 0].values.ravel()
y_test = y[isTest[0] == 1].values.ravel()
X_train.shape, X_test.shape

((2992, 4096), (340, 4096))

In [59]:
from time import time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

n_components = 150

t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("fit done in %0.3fs" % (time() - t0))

t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("transform done in %0.3fs" % (time() - t0))


t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'),
                   param_grid, cv=5)
clf = clf.fit(X_train_pca, y_train)
print("train done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

fit done in 2.027s
transform done in 0.107s
train done in 333.448s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [60]:
y_pred = clf.predict(X_test_pca)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         5
           9       0.71      1.00      0.83         5
          10       1.00      1.00      1.00         5
          11       1.00      1.00      1.00         5
          12       1.00      1.00      1.00         5
          13       1.00      1.00      1.00         5
          14       1.00      1.00      1.00         5
          15       1.00      1.00      1.00         5
          16       0.83      1.00      0.91         5
          17       1.00      1.00      1.00         5
          18       1.00    